In [10]:
import praw
import pandas as pd

reddit = praw.Reddit(client_id='I9PpbjO90sxduFVaZd2G8A', client_secret='Gi9Etic753bP8rbVWcM_omx8Kv3x9Q', user_agent='261662WebScraping')

topics = ['gaming', 'technology', 'programming']

users = {'Usernames': [], 'Topics': []}

for topic in topics:
    subreddit = reddit.subreddit(topic)
    for submission in subreddit.top(limit=100):
        author_name = submission.author.name if submission.author else None
        if author_name is not None:
            users['Usernames'].append(author_name)
            users['Topics'].append(topic)

df = pd.DataFrame(users)
df


,Usernames,Topics
0,SrGrafo,gaming
1,Yabba_Dabbs,gaming
2,springfart,gaming
3,velabas,gaming
4,OtterDeity,gaming
...,...,...
281,mzaiady,programming
282,jailbird,programming
283,zbhoy,programming
284,whackri,programming


In [13]:
import praw
import pandas as pd

reddit = praw.Reddit(client_id='I9PpbjO90sxduFVaZd2G8A', client_secret='Gi9Etic753bP8rbVWcM_omx8Kv3x9Q', user_agent='261662WebScraping')

def get_top_comment(submission):
    top_comment = None
    top_score = float('-inf')

    for comment in submission.comments:
        if isinstance(comment, praw.models.Comment) and comment.score > top_score:
            top_score = comment.score
            top_comment = comment.body

    return top_comment

def get_post_data(submission, topic):
    post_info = {
        'Topic': topic,
        'Title': submission.title,
        'Author': submission.author.name if submission.author else None,
        'Time of creation': pd.to_datetime(submission.created_utc, unit='s'), 
        'Score': submission.score,
        'Num comments': submission.num_comments,
        'Shares': submission.num_crossposts,
        'Top_comment_author': None,
        'Top_comment_body': None,
        'Top_comment_score': float('-inf'),
        'Time of creation comment': None
    }

    for comment in submission.comments:
        if isinstance(comment, praw.models.Comment) and comment.score > post_info['Top_comment_score']:
            post_info['Top_comment_author'] = comment.author.name if comment.author else None
            post_info['Top_comment_body'] = comment.body
            post_info['Top_comment_score'] = comment.score
            post_info['Time of creation comment'] = pd.to_datetime(comment.created_utc, unit='s') 

    return post_info

topics = ['gaming', 'technology', 'programming']

posts_data = []

for topic in topics:
    subreddit = reddit.subreddit(topic)
    for submission in subreddit.top(limit=100):
        author_name = submission.author.name if submission.author else None
        if author_name is not None:
            post_data = get_post_data(submission, topic)
            posts_data.append(post_data)

df = pd.DataFrame(posts_data)
df


,Topic,Title,Author,Time of creation,Score,Num comments,Shares,Top_comment_author,Top_comment_body,Top_comment_score,Time of creation comment
0,gaming,"Take your time, you got this",SrGrafo,2019-07-13 16:28:13,269748,3568,151,GeekyMeerkat,Them: I don't think I like this game.\n\nMe: B...,23056,2019-07-13 16:30:04
1,gaming,My parents (late 70s) got me a ps5 controller ...,Yabba_Dabbs,2020-12-25 19:07:30,241202,5501,15,Yabba_Dabbs,Just realized there's full steam support. Game...,29581,2020-12-25 20:20:55
2,gaming,I got off the horse by accident right before a...,springfart,2018-12-01 01:37:22,227166,2193,80,Garrus127,I have no idea why this is so funny to watch. ...,22615,2018-12-01 01:47:13
3,gaming,boy gamer,velabas,2021-03-01 15:41:09,220472,6548,38,ReigningTierney,Oh you're a guy huh? I don't believe you. Mess...,16983,2021-03-01 16:28:33
4,gaming,"Minesweeper 99 x 99, 1500 mines. Took me about...",OtterDeity,2022-02-14 16:52:26,202919,2754,13,corbone30,POV you are a Bosnian child in the woods,9107,2022-02-14 16:55:20
...,...,...,...,...,...,...,...,...,...,...,...
281,programming,'Kernel memory leaking' Intel processor design...,mzaiady,2018-01-03 05:13:19,5919,1093,4,vonKemper,The performance impact this is going to have o...,670,2018-01-03 06:10:36
282,programming,"An insane answer to ""What's the largest amount...",jailbird,2018-11-14 13:49:43,5883,1158,3,thebritisharecome,"I'm a software engineer contractor, we're usua...",2873,2018-11-14 14:44:47
283,programming,"Microsoft unveils Windows Terminal, a new comm...",zbhoy,2019-05-06 17:17:43,5846,1082,3,uzimonkey,First Notepad finally understands different li...,1967,2019-05-06 18:01:48
284,programming,Linus Torvalds rails against 80-character-line...,whackri,2021-01-03 18:42:04,5828,1163,1,IanSan5653,"I like 100 or 120, as long as it's consistent....",1708,2021-01-03 19:24:16


In [25]:
import praw
import pandas as pd
from bs4 import BeautifulSoup
import requests
reddit = praw.Reddit(client_id='I9PpbjO90sxduFVaZd2G8A', client_secret='Gi9Etic753bP8rbVWcM_omx8Kv3x9Q', user_agent='261662WebScraping')

def get_user_metrics(username):
    try:
        user = reddit.redditor(username)
        followers = get_followers_count(username)
        
        return pd.DataFrame([{
            'Username': username,
            # 1 Ocena popularności - liczba komentarzy
            'Total Comments': len(list(user.comments.new(limit=None))),
            # 2 Ocena popularności - liczba wpisów
            'Total Posts': len(list(user.submissions.new(limit=None))),
            # 3 Ocena popularności - karma użytkownika
            'Karma': user.link_karma + user.comment_karma,
            'Cake Day': pd.to_datetime(user.created_utc, unit='s', utc=True).strftime('%Y-%m-%d'),
            # 4 Ocena popularności - followersi
            'Followers': followers
    }])
    except:
        return pd.DataFrame([{}])
    

def get_followers_count(username):
    url = f'https://www.reddit.com/user/{username}/'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    followers_count = 0
    followers_element = soup.find('span', {'class': 'u-follower-count'})
    if followers_element:
        followers_count = int(followers_element.text.replace(',', ''))

    return followers_count

def get_user_posts_and_comments(username):
    try:
        user = reddit.redditor(username)       
        user_posts = []
        for submission in user.submissions.new(limit=5):
            post_data = {
                'Title latest 5 posts': submission.title,
                'Likes of post': submission.score,
                'Post number of comment': submission.num_comments,
                'Shares': submission.num_crossposts,
            }
            

            comments = list(submission.comments)
            if comments:
                top_comment = comments[0]
                post_data['Author Latest Comment of Latest Post'] = top_comment.author.name if top_comment.author else None
                post_data['Title Latest Comment of Latest Post'] = top_comment.body
                post_data['Score - likes of comment of Latest Post'] = top_comment.score

            user_posts.append(post_data)

        user_comments = []
        for comment in user.comments.new(limit=5):
            comment_data = {
                'Users Comment': comment.body,
                'Users Likes of comment': comment.score,
                'Users Time Created comment': pd.to_datetime(comment.created_utc, unit='s', utc=True).strftime('%Y-%m-%d %H:%M:%S')
            }
            user_comments.append(comment_data)

        return pd.DataFrame(user_posts), pd.DataFrame(user_comments)
    except:
        return pd.DataFrame([{}]), pd.DataFrame([{}])

topics = ['gaming', 'technology', 'programming']

all_users_details = []

for topic in topics:
    subreddit = reddit.subreddit(topic)
    for submission in subreddit.top(limit=100):
        author_name = submission.author.name if submission.author else None
        if author_name:
            user_metrics = get_user_metrics(author_name)
            user_posts, user_comments= get_user_posts_and_comments(author_name)
            user_data = pd.merge(user_metrics, user_posts, how='outer', left_index=True, right_index=True)
            user_data = pd.merge(user_data, user_comments, how='outer', left_index=True, right_index=True)
            
            all_users_details.append(user_data)

result_df = pd.concat(all_users_details, ignore_index=True)

result_df = result_df.fillna('')
result_df


,Username,Total Comments,Total Posts,Karma,Cake Day,Followers,Title latest 5 posts,Likes of post,Post number of comment,Shares,Author Latest Comment of Latest Post,Title Latest Comment of Latest Post,Score - likes of comment of Latest Post,Users Comment,Users Likes of comment,Users Time Created comment
0,SrGrafo,1000.0,1002.0,14969119.0,2012-06-18,0.0,Bee Angry hoodie is out now !,2773.0,40.0,0.0,SrGrafo,[**Hoodie available here**](https://srgrafo.my...,1.0,[source](https://twitter.com/kokooooo_EX/statu...,2.0,2023-11-19 17:02:30
1,,,,,,,Zoe #932,4625.0,102.0,0.0,SrGrafo,"*Doing Zoe's hair always take ages, and I didn...",226.0,its in my todo list once I get more time u\_u,15.0,2023-11-02 16:42:47
2,,,,,,,SrGrafo's Projects,8707.0,138.0,0.0,SrGrafo,r/RimworldTales Story set on the Rimworld univ...,1.0,"enjoy it while it last, it wont happen again >:I",37.0,2023-11-02 16:39:30
3,,,,,,,edIT,19697.0,817.0,1.0,Miss_Nameless,Uhhh... That's creepy. I'd like to grab my edi...,1984.0,"wait, wtf how >:0",43.0,2023-11-02 16:35:56
4,,,,,,,by @kokooooo_EX,40.0,2.0,0.0,SrGrafo,[source](https://twitter.com/kokooooo_EX/statu...,3.0,"I can see why you thought I was a feet artist,...",39.0,2023-11-02 16:35:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,drizzcool,430.0,45.0,9712.0,2013-12-04,0.0,Leg for Skorva Midbeam?,0.0,7.0,0.0,Confident_Resolution,Skorva is galvanised steel - not sure what you...,4.0,Have you contacted Sony?,1.0,2023-10-12 11:26:21
1362,,,,,,,It takes one hour to leave this suburb by car ...,26.0,3.0,0.0,pedz,This is a two fold issue. \n\nOn one side ther...,6.0,Are you still having this issue?,1.0,2023-08-25 12:27:01
1363,,,,,,,@ljayes: Why the government is letting vaccine...,32.0,15.0,0.0,,I wonder how this political play by Morrison w...,4.0,This is happening for me too with the Sigma 56...,1.0,2023-08-25 11:51:36
1364,,,,,,,Melbourne to exit 112-day lockdown,10.0,7.0,0.0,DrFeelLikeMyInsides,Knowing people in Melbourne they’ll fuck this ...,2.0,Great idea. I'll give that a go. Thanks,2.0,2023-05-09 21:29:53
